In [ ]:
import pandas as pd

class ModelFlaggingSystem:
    def __init__(self, overfitting_threshold=0.1, underfitting_threshold=0.5):
        """
        Initializes the flagging system with thresholds for overfitting and underfitting.
        
        Parameters:
            overfitting_threshold (float): The acceptable difference between train and validation loss for overfitting.
            underfitting_threshold (float): The maximum acceptable validation loss for underfitting.
        """
        self.overfitting_threshold = overfitting_threshold
        self.underfitting_threshold = underfitting_threshold
        self.flagged_combinations = []

    def flag_model(self, selected_indicators, train_loss, val_loss):
        """
        Flags the model based on the training and validation loss.
        
        Parameters:
            selected_indicators (list): The list of selected indicators for the model.
            train_loss (float): The final training loss.
            val_loss (float): The final validation loss.
        """
        loss_diff = abs(train_loss - val_loss)

        if loss_diff > self.overfitting_threshold:
            self.flagged_combinations.append({
                'Indicators': ', '.join(selected_indicators),
                'Status': 'Overfitting',
                'Train Loss': train_loss,
                'Validation Loss': val_loss,
                'Loss Difference': loss_diff
            })
        elif val_loss > self.underfitting_threshold:
            self.flagged_combinations.append({
                'Indicators': ', '.join(selected_indicators),
                'Status': 'Underfitting',
                'Train Loss': train_loss,
                'Validation Loss': val_loss,
                'Loss Difference': loss_diff
            })

    def get_flagged_combinations(self):
        """
        Returns the list of flagged combinations.
        
        Returns:
            pd.DataFrame: DataFrame containing the flagged model combinations.
        """
        return pd.DataFrame(self.flagged_combinations)

    def save_flagged_combinations(self, file_name="flagged_combinations.csv"):
        """
        Saves the flagged combinations to a CSV file.
        
        Parameters:
            file_name (str): The name of the CSV file to save the flagged combinations.
        """
        flagged_df = self.get_flagged_combinations()
        flagged_df.to_csv(file_name, index=False)
        print(f"Flagged combinations saved to {file_name}")
    
    def plot_flagged_combinations(self):
        """
        Plots the training and validation loss for flagged models.
        """
        import matplotlib.pyplot as plt

        # Plot the losses for all combinations
        for selected_indicators in self.get_flagged_combinations()['Indicators'].unique():
            # Filter the data for the current set of indicators
            subset = self.get_flagged_combinations()[self.get_flagged_combinations()['Indicators'] == selected_indicators]
            
            # Plot the training and validation loss for flagged combinations
            plt.plot(subset['Epoch'], subset['Train Loss'], label=f'Train Loss - {selected_indicators}')
            plt.plot(subset['Epoch'], subset['Val Loss'], label=f'Val Loss - {selected_indicators}')
            
            # Highlight flagged points
            if subset['Status'].iloc[0] == 'Overfitting':
                plt.scatter(subset['Epoch'], subset['Train Loss'], color='red', label='Overfitting')
            elif subset['Status'].iloc[0] == 'Underfitting':
                plt.scatter(subset['Epoch'], subset['Val Loss'], color='blue', label='Underfitting')

        plt.title("Training vs Validation Loss with Flagged Models")
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

